# Chapter 5 - Person Matching

In [ ]:
import requests
import json
import zipfile
import io
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

## Step 1 - Data Acquisition

### Wikipedia

In [ ]:
# Get the Wikipedia webpage and find all tables.

url = "https://en.wikipedia.org/wiki/List_of_MPs_elected_in_the_2019_United_Kingdom_general_election"

website_page = requests.get(url).text
soup = BeautifulSoup(website_page,'html.parser')
tables = soup.find_all('table')

In [ ]:
# Find the table with members returned, extract rows as a list of lists and load into dataframe

for table in tables:
    if 'Member returned' in table.text:
        headers = [header.text.strip() for header in table.find_all('th')]
        headers = headers[:5]
        dfrows = []
        table_rows = table.find_all('tr')    
        for row in table_rows:
            td = row.find_all('td')
            dfrow = []
            for element in td:
                if element.text!='\n':
                    dfrow.append(element.text)
                if element.select("a[class='image']"):
                    for link in element.select("a[title]"):
                        urltail = link['href']
                        url = f'https://en.wikipedia.org{urltail}'
            dfrow.append(url)
            dfrows.append(dfrow)
        headers.append('Wikilink')
df_w = pd.DataFrame(dfrows, columns=headers)
df_w = df_w.dropna()

In [ ]:
def get_bday(url):
    wiki_page = requests.get(url).text
    soup = BeautifulSoup(wiki_page,'html.parser')
    bday = ''
    bdayelement = soup.select_one("span[class='bday']")
    if bdayelement is not None:
        bday = bdayelement.text
    return(bday)

In [ ]:
df_w['Birthday'] = df_w.apply(lambda x: get_bday(x.Wikilink), axis=1)

In [ ]:
df_w.head(n=5)

### Companies House Persons of Significant Control

In [ ]:
# UK Companies House Persons with Significant Control Download Page

url = "http://download.companieshouse.gov.uk/en_pscdata.html"

In [ ]:
# Download snapshots, convert json to dataframe
# Ensure all necessary columns are present in each snapshot dataframe
# Drop rows if any of: company number, surname, forename, month of birth or year of birth are missing
# Append to a single dataframe

df_psctotal = pd.DataFrame()
with requests.Session() as req:
        r = req.get(url)
        soup = BeautifulSoup(r.content, 'html.parser')
        snapshots = [f"{url[:38]}{item['href']}" for item in soup.select(
              "a[href*='psc-snapshot']")]
        for snapshot in snapshots:   
            print(snapshot)
            response = requests.get(snapshot).content     
            zipsnapshot = zipfile.ZipFile(io.BytesIO(response))         
            tempfile = zipsnapshot.extract(zipsnapshot.namelist()[0])
            df_psc = pd.json_normalize(pd.Series(open(tempfile, encoding="utf8").readlines()).apply(json.loads))
                
            must_cols = ['company_number','data.name_elements.surname','data.name_elements.middle_name','data.name_elements.forename','data.date_of_birth.month','data.date_of_birth.year','data.name_elements.title','data.nationality']
            all_cols =list(set(df_psc.columns).union(must_cols))
            df_psc=df_psc.reindex(columns=sorted(all_cols))
              
            df_psc = df_psc.dropna(subset=['company_number','data.name_elements.surname','data.name_elements.forename','data.date_of_birth.month','data.date_of_birth.year'])       
            df_psc = df_psc[must_cols] 
            df_psctotal = pd.concat([df_psctotal, df_psc], ignore_index=True)

### Saving to Local Storage

In [ ]:
df_w.to_csv('mps_wiki_bday_raw.csv', index=False)
df_psctotal.to_csv('psc_raw.csv', index=False)

## Step 2 - Data Standardization

In [ ]:
df_w = pd.read_csv('mps_wiki_bday_raw.csv')
df_psc = pd.read_csv('psc_raw.csv',dtype={'data.name_elements.surname':'string',
                                           'data.name_elements.forename':'string',
                                           'data.name_elements.middle_name':'string',
                                           'data.name_elements.title':'string',
                                           'data.nationality':'string'})

### Wikipedia

In [ ]:
df_w = df_w.dropna()
df_w['Year'] = pd.to_datetime(df_w['Birthday']).dt.year.astype('int64')
df_w['Month'] = pd.to_datetime(df_w['Birthday']).dt.month.astype('int64')

In [ ]:
df_w = df_w.rename(columns={ 'Member returned' : 'Fullname'})
df_w['Fullname'] = df_w['Fullname'].str.rstrip("\n")
df_w['Fullname'] = df_w['Fullname'].str.lstrip("\n")
df_w['Firstname'] = df_w['Fullname'].str.split().str[0]
df_w['Lastname'] = df_w['Fullname'].astype(str).apply(lambda x: ' '.join(x.split()[1:]))

In [ ]:
df_w['unique_id'] = df_w.index
df_w["company_number"] = np.nan
df_w=df_w[['Firstname','Lastname','Month','Year','unique_id','company_number']]

In [ ]:
df_w.head(n=5)

### Companies House Persons of Significant Control

In [ ]:
# Rename and convert columns for matching

df_psc = df_psc.dropna(subset=['data.date_of_birth.year', 'data.date_of_birth.month'])

df_psc['Year'] = df_psc['data.date_of_birth.year'].astype('int64')
df_psc['Month'] = df_psc['data.date_of_birth.month'].astype('int64')
df_psc['Firstname']=df_psc['data.name_elements.forename']
df_psc['Lastname']=df_psc['data.name_elements.surname']

# Create unique index column needed by Splink from dataframe index
# Subset down to required columns

df_psc['unique_id'] = df_psc.index
df_psc = df_psc[['Lastname','Firstname','company_number','Year','Month','unique_id']]


### Saving to Local Storage

In [ ]:
#df_w.to_csv('mps_wiki_bday_clean.csv', index=False)
df_w = pd.read_csv('mps_wiki_bday_clean.csv')
#df_psc.to_csv('psc_clean.csv', index=False)
df_psc = pd.read_csv('psc_clean.csv')

# Step 3 - Record Blocking and Attribute Comparison

In [ ]:
len(df_psc)

In [ ]:
# Count of simple merge on matching Year and Month 

df_mp = df_w.merge(df_psc, on=['Year','Month'], suffixes=('_w', '_psc'))
len(df_mp)

In [ ]:
# Calculate exact match using a simple join

df_result = df_w.merge(df_psc, on=['Lastname','Firstname','Year','Month'], suffixes=('_w', '_psc'))
df_result

In [ ]:
# Splink settings to block on year and month matches and then compare First and Last names

from splink.duckdb.linker import DuckDBLinker
from splink.duckdb import comparison_library as cl
settings = {
    "link_type": "link_only",
    "blocking_rules_to_generate_predictions": [
        "l.Year = r.Year and l.Month = r.Month"
    ],
    "comparisons": [
        cl.jaro_winkler_at_thresholds("Firstname", [0.9]),
        cl.jaro_winkler_at_thresholds("Lastname", [0.9]),
        cl.exact_match("Month"),
        cl.exact_match("Year", term_frequency_adjustments=True),
    ],
    "additional_columns_to_retain": ["company_number"]
}

In [ ]:
# Setup linker and profile columns

linker = DuckDBLinker([df_w, df_psc], settings, input_table_aliases=["df_w", "df_psc"])
linker.profile_columns(["Firstname","Lastname","Month","Year"], top_n=10, bottom_n=5)

In [ ]:
# Estimate u values

linker.estimate_u_using_random_sampling(max_pairs=1e7)

In [ ]:
## Calculate m values 

linker.estimate_parameters_using_expectation_maximisation("l.Lastname = r.Lastname and l.Month = r.Month", fix_u_probabilities=False)
linker.estimate_parameters_using_expectation_maximisation("l.Firstname = r.Firstname and  l.Year = r.Year", fix_u_probabilities=False)

In [ ]:
#linker.save_model_to_json("Chapter5_Splink_Settings.json", overwrite=True)
linker.load_settings("Chapter5_Splink_Settings.json")

In [ ]:
linker.match_weights_chart()

In [ ]:
linker.m_u_parameters_chart()

# Step 4 - Match Classification

In [ ]:
# Predict matches and convert to dataframe

results = linker.predict(threshold_match_probability=0.99)
pres = results.as_pandas_dataframe()
pres = pres.rename(columns={"Firstname_l": "Firstname_psc", 
                            "Lastname_l": "Lastname_psc",
                            "Firstname_r":"Firstname_w",
                            "Lastname_r":"Lastname_w",
                            "company_number_l":"company_number"})
pres = pres[['match_weight','match_probability','Firstname_psc','Firstname_w', 'Lastname_psc','Lastname_w','company_number','unique_id_r']]
pres

In [ ]:
# Select matches that aren't exact

pres[(pres['Lastname_psc']!=pres['Lastname_w']) | (pres['Firstname_psc']!=pres['Firstname_w'])]

# Chapter 10 Extension 

In [ ]:
df_pres = pres.groupby(['unique_id_r'], sort=False).agg(lambda x: list(set(x))).reset_index()
df_pres[(df_pres['company_number'].apply(len)>2) & df_pres['Lastname_psc'].apply(len)==1]